
# ΕΡΓΑΣΙΑ "ΑΝΑΛΥΣΗ ΚΑΙ ΔΙΑΧΕΙΡΙΣΗ ΧΑΡΤΟΦΥΛΑΚΙΟΥ"
### ΚΟΥΚΟΥΛΑΡΗΣ ΝΙΚΟΛΑΟΣ (7190300) 
### ΤΜΗΜΑ ΛΟΓΙΣΤΙΚΗΣ ΚΑΙ ΧΡΗΜΑΤΟΟΚΟΝΟΜΙΚΗΣ (ΟΙΚΟΝΟΜΙΚΟ ΠΑΝΕΠΙΣΤΗΜΙΟ ΑΘΗΝΩΝ)

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from functools import reduce
import numpy as np

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

from pypfopt import HRPOpt

from pypfopt.efficient_frontier import EfficientCVaR

## Άντληση δεδομένων απο το API του YahooFinance.com
Συγκεκτιμένα αντλούμε δεδομένα κλεισίματος (Close) απο την ημερομηνία 1/1/2017 (start) εως αρχες του έτους 2020 (end) και αποθηκεύμε στη μεταβλητή data. Στην συνέχεια ορίζουμε μια συνάρτηση (combine stocks) με απότερο σκοπό να συνδιάσουμε τα χρεόγραφα μας βάση των tickers τους και στο τέλος επιστρέφει το (df merged) όπου κατανέμει τα δεδομένα όπου αντλήσαμε. Τέλος, αποθηκεύουμε σε μια λίστα (stocks) όλα τα σύμβολα (tickers) των μετοχών που θα πραγματοποιήσουμε την ανάλυση. 
Η λίστα portofolio εκτελεί τη συνάρτιση get_stock η οποία πέρνει ως όρισμα τη λίστα stocks.

In [2]:

start = dt.datetime(2019,1,1)
end = dt.datetime(2021,1,1)

def get_stock(ticker):
    data = web.DataReader(ticker,"yahoo",start,end)
    data[ticker] = data["Close"]
    data = data[[ticker]] 
    # print(data.head())
    return data 


def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='outer'), data_frames)
    print(df_merged.head())
    return df_merged


stocks = ["BTC-USD", "XRP-USD", "ETH-USD", "GOOGL", 
          "FB", "AAPL", "COST", "WMT", "KR", "JPM", 
          "BAC", "HSBC"]
portfolio = combine_stocks(stocks)


PORTFOLIO_CAPITAL = 100000



                BTC-USD   XRP-USD     ETH-USD        GOOGL          FB  \
Date                                                                     
2019-01-01  3843.520020  0.364771  140.819412          NaN         NaN   
2019-01-02  3943.409424  0.375243  155.047684  1054.680054  135.679993   
2019-01-03  3836.741211  0.360224  149.135010  1025.469971  131.740005   
2019-01-04  3857.717529  0.356747  154.581940  1078.069946  137.949997   
2019-01-05  3845.194580  0.355275  155.638596          NaN         NaN   

                 AAPL        COST        WMT         KR         JPM  \
Date                                                                  
2019-01-01        NaN         NaN        NaN        NaN         NaN   
2019-01-02  39.480000  204.759995  93.339996  27.299999   99.309998   
2019-01-03  35.547501  200.419998  92.860001  27.350000   97.110001   
2019-01-04  37.064999  206.240005  93.440002  27.660000  100.690002   
2019-01-05        NaN         NaN        NaN        NaN

### Αποθηκεύουμε το portfolio σε ενα αρχείο csv ,με μόνη διαφορά οτι θέλουμε να αφαιρέσει τις ημερομηνίες (index) 
Ο λόγος για τον οποίο αποθήκεύουμε σε csv είναι διότι δεν θέλουμε να ζητάμε τα 
δεδομένα απο το API του YahooFinance, καθώς αυτό είναι χρονοβόρο 

In [3]:
portfolio.to_csv("portfolio.csv", index=False)


In [4]:
portfolio = pd.read_csv("portfolio.csv")
portfolio

,BTC-USD,XRP-USD,ETH-USD,GOOGL,FB,AAPL,COST,WMT,KR,JPM,BAC,HSBC
0,3843.520020,0.364771,140.819412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3943.409424,0.375243,155.047684,1054.680054,135.679993,39.480000,204.759995,93.339996,27.299999,99.309998,24.959999,40.880001
2,3836.741211,0.360224,149.135010,1025.469971,131.740005,35.547501,200.419998,92.860001,27.350000,97.110001,24.559999,40.430000
3,3857.717529,0.356747,154.581940,1078.069946,137.949997,37.064999,206.240005,93.440002,27.660000,100.690002,25.580000,41.610001
4,3845.194580,0.355275,155.638596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
728,27362.437500,0.220962,731.520142,1757.760010,276.779999,134.869995,372.720001,144.300003,31.459999,125.010002,30.010000,25.820000
729,28840.953125,0.211828,751.618958,1736.250000,271.869995,133.720001,374.450012,144.179993,31.549999,125.360001,29.980000,25.940001
730,29001.720703,0.219846,737.803406,1752.640015,273.160004,132.690002,376.779999,144.149994,31.760000,127.070000,30.309999,25.910000
731,29374.152344,0.237444,730.367554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Mean Variance optimization
H πρώτη μέθοδος που θα προσεγκίσουμε είναι μέθοδος "Βελτιστοποίησης Μέσου-Διακύμανσης" που χρησιμοποιούμε το υπόδειγμα του Harry Markowitz για την οποία θα χρειαστόυμε να υπολογίσουμε το ιστορικό μέσο (mean) των αποδόσεων των μετοχών που απαρτίζεται το χαρτοφυλάκιο και την συνδιακύμανση αυτών (covariance matrix) αποθηκεύοντας τα στα data frames mu και S αντίστοιχα.

In [5]:
mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()
S

,BTC-USD,XRP-USD,ETH-USD,GOOGL,FB,AAPL,COST,WMT,KR,JPM,BAC,HSBC
BTC-USD,0.333888,0.259673,0.333874,0.031174,0.040590,0.036733,0.018380,0.019927,0.007393,0.029516,0.033790,0.020608
XRP-USD,0.259673,0.612987,0.384048,0.031472,0.040483,0.040561,0.022041,0.024440,0.011304,0.035867,0.042398,0.020878
ETH-USD,0.333874,0.384048,0.505920,0.044432,0.049298,0.052252,0.027771,0.033548,0.011509,0.038992,0.045499,0.023085
GOOGL,0.031174,0.031472,0.044432,0.075238,0.058865,0.055951,0.027675,0.021889,0.011851,0.047154,0.052293,0.030129
FB,0.040590,0.040483,0.049298,0.058865,0.102691,0.065660,0.028546,0.021149,0.012619,0.043871,0.049441,0.026515
AAPL,0.036733,0.040561,0.052252,0.055951,0.065660,0.102591,0.035295,0.028121,0.015580,0.052354,0.059149,0.032883
COST,0.018380,0.022041,0.027771,0.027675,0.028546,0.035295,0.044574,0.026767,0.018148,0.024170,0.026830,0.014253
WMT,0.019927,0.024440,0.033548,0.021889,0.021149,0.028121,0.026767,0.047671,0.021198,0.021229,0.024126,0.011157
KR,0.007393,0.011304,0.011509,0.011851,0.012619,0.015580,0.018148,0.021198,0.068398,0.009055,0.013411,0.004336
JPM,0.029516,0.035867,0.038992,0.047154,0.043871,0.052354,0.024170,0.021229,0.009055,0.117175,0.112379,0.058410


In [6]:
mu

BTC-USD    1.077111
XRP-USD   -0.157594
ETH-USD    0.798392
GOOGL      0.191348
FB         0.272814
AAPL       0.518758
COST       0.233957
WMT        0.161629
KR         0.053550
JPM        0.088689
BAC        0.069240
HSBC      -0.145469
dtype: float64

## 1.1 Αποτελεσματικό Σύνορο και Ποσοστά επένδυσης σε κάθε χρεόγραφο (W1-Wn)
Με την χρήση της συνάρτησης EfficientFrontier που πέρνει ως όρισμα τα mu και S (τα οποία υπολογίσαμε νωρίτερα ) μας δίνεται η δυνατότητα να υπολογίσουμε το αποτελεσματικό σύνορο χαρτοφυλακίων και στη συνέχεια τα w (weights) ,δηλαδή το ποσοστό (%) των διαθέσιμων κερφαλαίων που πρέπει να επενδύσουμε στην κάθε μετοχή κεχωριστά για να αποκαλείται Αποτελεσματικό Χαρτοφυλλάκιο. Στη συνέχεια εκτυπώνουμε τα αποτελέσματ των συναρτήσεων.

In [7]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights_mvo = ef.clean_weights()
print(dict(cleaned_weights_mvo))
ef

{'BTC-USD': 0.38098, 'XRP-USD': 0.0, 'ETH-USD': 0.0, 'GOOGL': 0.0, 'FB': 0.0, 'AAPL': 0.5038, 'COST': 0.11522, 'WMT': 0.0, 'KR': 0.0, 'JPM': 0.0, 'BAC': 0.0, 'HSBC': 0.0}


## 1.2 Απόδοση Χαρτοφυλλακίου (Performance)
Αποθηκεύουμε στη "mvo_exp_annual_ret" τις αναμενόμενες ετήσιες επιστροφές για κάθε επενδυτή με αυτό το χαρτοφυλάκιο, ομοίως στη "mvo_annual_volatility" τη ετήσια μεταβλητότητα - όγκο των μετοχών που περιέχει το χαρτοφυλάκιο μας και τέλος για την ευρεση του Sharpe Ratio αλλα και τον υπολογισμό του ,το αποθηκεύουμε στο "mvo_sharpe_r".Εκτελόντας αυτή τη γραμμή κώδικα θα μας επιστρέψει τα αποτελέσματα που ζητήσαμε.

In [8]:
mvo_exp_annual_ret, mvo_annual_volatility , mvo_sharpe_r = ef.portfolio_performance(verbose=True)

Expected annual return: 69.9%
Annual volatility: 30.8%
Sharpe Ratio: 2.20


## 1.3 Add Funds (How many stocks (n) according to the Funds)
Αφού πριν βρήκαμε τα ποσoτά των κεφαλαίων του κατόχου χαρτοφυλακίου που πρέπει να επενδυθούν (w), τώρα μπρορούμε να βρούμε τα χρήματα (σε δολάρια) που πρέπει να επενδύσουμε με γνώμονα οτι ο επενδυτής έχει στη διακριτική του ευκαίρια το ποσό των 100.000$. Για να πραγματοποιηθεί αυτό φτιάχνουμε μια μεταβλητή allocation που στη συνέχεια την εκτυπώνουμε  ,δηλαδή διανομή των χρημάτων ως που να είναι αποτελεσματική και κερδοφόρα η τοποθετησή τους σε ενα αξιόγραφο του χαρτοφυλακίου και στο τέλος εκτυπώνει τα υπολοιπόμενα λεφτά του επενδυτή που παρέμηναν εκτός λόγω του οτι δεν απαιτούνται για τη δημιουργία αποτελεσματικού χαρτοφυλακίου.

In [9]:
latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=PORTFOLIO_CAPITAL)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'AAPL': 379, 'BTC-USD': 1, 'COST': 30}
Funds remaining: $6279.82


## 2. Hierarchical Risk Parity
Στη δεύτερη μέθοδο όπου θα κάνουμε ανάλυση δε απαιτείται ο υπολογισμός του πίνακα συνδιακίμανσης όπως κάναμε στη πρώτη μέθοδο. Για αρχή θα εκτυπώσουμε τις αποδόσεις των χρεογράφων του χαρτοφυλλακίου με τις οποίες θα ασχοληθούμε στη συνέχεια.

In [10]:
returns = portfolio.pct_change().dropna()
returns

,BTC-USD,XRP-USD,ETH-USD,GOOGL,FB,AAPL,COST,WMT,KR,JPM,BAC,HSBC
2,-0.027050,-0.040025,-0.038135,-0.027696,-0.029039,-0.099607,-0.021196,-0.005142,0.001832,-0.022153,-0.016026,-0.011008
3,0.005467,-0.009652,0.036523,0.051294,0.047138,0.042689,0.029039,0.006246,0.011335,0.036865,0.041531,0.029186
4,-0.003246,-0.004126,0.006836,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.060189,0.036929,0.013542,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,-0.012605,-0.010988,-0.038334,-0.001994,0.000725,-0.002226,0.003685,0.011772,0.009400,0.000695,-0.000782,-0.013458
...,...,...,...,...,...,...,...,...,...,...,...,...
728,0.010250,-0.108873,0.001537,-0.009132,-0.000794,-0.013315,0.004474,-0.006335,0.002230,-0.002633,-0.003983,-0.001161
729,0.054034,-0.041337,0.027475,-0.012237,-0.017740,-0.008527,0.004642,-0.000832,0.002861,0.002800,-0.001000,0.004648
730,0.005574,0.037851,-0.018381,0.009440,0.004745,-0.007703,0.006222,-0.000208,0.006656,0.013641,0.011007,-0.001157
731,0.012842,0.080047,-0.010078,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## W1-Wn
Στη πρώτη μέθοδο χρειάστεικε η εύρεση του Αποτελεσματικού Συνόρου των χαρτοφυλλακίων ,με τη παρούσα μέθοδο θα βρούμε τα ποσοστά (w) που πρέπει να επενδυθούν με την χρήση του αλγορίθμου optimize και στη συνέχεια τα εκτυπώνουμε.

In [11]:
hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()
hrp_weights

OrderedDict([('AAPL', 0.07068989527822178),
             ('BAC', 0.06920526586235945),
             ('BTC-USD', 0.017327142092896715),
             ('COST', 0.14550697916508787),
             ('ETH-USD', 0.011406401678718817),
             ('FB', 0.04087932492805997),
             ('GOOGL', 0.05758011798330394),
             ('HSBC', 0.1289301135575259),
             ('JPM', 0.07641190343195456),
             ('KR', 0.23101817167449268),
             ('WMT', 0.13413299595399847),
             ('XRP-USD', 0.016911688393379896)])

## 2.1 Aπόδοση του χαρτοφυλλακίου (Performance)
Αποθηκεύουμε στη "hrp_exp_annual_ret" τις αναμενόμενες ετήσιες επιστροφές για κάθε επενδυτή με αυτό το χαρτοφυλλάκιο, ομοίως στη "hrp_annual_volatility" τη ετήσια μεταβλητότητα - όγκο των μετοχών που περιέχει το χαρτοφυλλακιό μας και τέλος για την ευρεση του Sharpe Ratio αλλα και τον υπολογισμό του ,το αποθηκεύουμε στο "hrp_sharpe_r".Εκτελόντας αυτή τη γραμμή κώδικα θα μας επιστρέψει τα αποτελέσματα που ζητήσαμε και ξανάεκτυπώνουμε τα απαιτούμενα weights ξανά για λόγους υπενθύμισης.

In [12]:
hrp_exp_annual_ret, hrp_annual_volatility, hrp_sharpe_r = hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

Expected annual return: 16.4%
Annual volatility: 17.9%
Sharpe Ratio: 0.81
{'AAPL': 0.07068989527822178, 'BAC': 0.06920526586235945, 'BTC-USD': 0.017327142092896715, 'COST': 0.14550697916508787, 'ETH-USD': 0.011406401678718817, 'FB': 0.04087932492805997, 'GOOGL': 0.05758011798330394, 'HSBC': 0.1289301135575259, 'JPM': 0.07641190343195456, 'KR': 0.23101817167449268, 'WMT': 0.13413299595399847, 'XRP-USD': 0.016911688393379896}


## 2.2 Add Funds (How many stocks (n) according to the Funds)
Χωρίς να χρησιμοποιήσουμε τις τρέχουσες τιμές όπως στη πρώτη μέθοδο ,μπρορούμε να βρούμε τα χρήματα (σε δολάρια) που πρέπει να επενδύσουμε με γνώμονα οτι ο επενδυτής έχει στη κατοχή του το ποσό των 100.000$. Για να πραγματοποιηθεί αυτό φτιάχνουμε μια μεταβλητή allocation που στη συνέχεια την εκτυπώνουμε  ,δηλαδή διανομή των χρημάτων ως που να είναι αποτελεσματική και κερδοφόρα η τοποθετησή τους σε ενα αξιόγραφο του χαρτοφυλλακίου και στο τέλος εκτυπώνει τα υπολοιπόμενα λεφτά (leftover) του επενδυτή που παρέμηναν εκτός λόγω του οτι δεν απαιτούνται για τη δημιουργία αποτελεσματικού χαρτοφυλλακίου.

In [13]:
da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=PORTFOLIO_CAPITAL)

allocation, leftover = da_hrp.greedy_portfolio()
print("Discrete allocation (HRP):", allocation)
print("Funds remaining (HRP): ${:.2f}".format(leftover))


Discrete allocation (HRP): {'KR': 727, 'COST': 39, 'WMT': 93, 'HSBC': 497, 'JPM': 60, 'AAPL': 53, 'BAC': 228, 'GOOGL': 4, 'FB': 15, 'XRP-USD': 7629, 'ETH-USD': 2}
Funds remaining (HRP): $17.35


## 3. Μέσος λαμβάνοντας υπόψη το Value at Risk (VaR)
Στη τρίτη και τελευταία μέθοδο που θα χρησιμοποιήσουμε πραγματοποιήται μια εκτίμηση στο χειρότερο σενάριο ο επενδυτής να έχει loss(έλλειμα) επενδύοντας σε αυτό το χαρτοφυλλάκιο. Δηλαδή δίνει την απάντηση στο εύλογο ερώτημα με ποία μετοχή θα χάσει τα περισσότερα χρήματα(πσοστά κεφαλαίων). Είναι μια εναλλακτική μορφή της πρώτης μεθόδου (Μεσου-Διακύμανσης). Επιστρέφουμε πάλι τα weights  και τα εκτυπώνουμε μέσω της χρήσης του αλγορίθμου το EfficientCVAR που πέρνει ως όρισμα τα mu και S απο την πρώτη μέθοδο.

In [14]:
S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()

cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

{'BTC-USD': 0.0, 'XRP-USD': 0.0, 'ETH-USD': 0.04599, 'GOOGL': 0.01942, 'FB': 0.0, 'AAPL': 0.0, 'COST': 0.0, 'WMT': 0.0, 'KR': 0.0, 'JPM': 0.9346, 'BAC': 0.0, 'HSBC': 0.0}


## Aπόδοση Χαρτοφυλλακίου (Performance)
Αποθηκεύουμε στη "cvar_exp_annual_ret" τις αναμενόμενες ετήσιες επιστροφές για κάθε επενδυτή που κατέχει αυτό το χαρτοφυλλάκιο, ομοίως στη "cvar_value_at_risk " τον κίνδυνο που ενέχει το χαρτοφυλλάκιο να μην επιστέψει τις αναμενόμενες επιστροφές στον επενδυτή. Δηλαδή ,μετράει τη φερεγγυότητα του χαρτοφυλλακίου.Με αυτή τη μέθοδο δεν υπολογίζουμε ούτε Sharpe Ratio αλλά ούτε το Volatility και τα αντικαστήσαμε με τον υπολοφισμό του VaR.

In [15]:
cvar_exp_annual_ret, cvar_value_at_risk = ef_cvar.portfolio_performance(verbose=True)

Expected annual return: 12.3%
Conditional Value at Risk: -46.95%


## 3.2 Add Funds (How many stocks (n) according to the Funds)
Με γνώμονα ότι ο επενδυτής διαθέτει το ποσό των 100.000$ ,πρέπει να διαμοιράσει τα κεφάλαια του όπου κρίνεται αποδοτικό, κερδοφόρο και να επιστρέψει πόσο απο το χρηματικό ποσό θα παραμείνει εκτός επένδυσης. Η συνάρτιση Allocation και leftover θα μας εκτυπώσουν αυτά τα αποτελέσματα.



In [16]:
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=PORTFOLIO_CAPITAL)

allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVAR):", allocation)
print("Funds remaining (CVAR): ${:.2f}".format(leftover))

Discrete allocation (CVAR): {'JPM': 735, 'ETH-USD': 6, 'GOOGL': 1, 'HSBC': 1, 'BAC': 1, 'KR': 1, 'XRP-USD': 1}
Funds remaining (CVAR): $115.50


## Real analysis of 2021, using the above methods
Παρακάτω δοκιμάζουμε τις τρείς μεθόδους αν πράγματι επιστρέφουν φερέγγυες τιμές. Απο οτι παρατηρούμε η πρώτη μέθοδος δεν είναι ακριβής σε σύγκριση με τις άλλες δύο.

In [17]:
start = dt.datetime(2021,1,1)
end = dt.datetime(2021,12,25)



In [18]:
# weights_dict = dict(cleaned_weights)
weights_dict = dict(cleaned_weights_mvo)
total_real_profit = 0
for ticker in weights_dict:
    one_stock = get_stock(ticker).values
    start_of_year_price = one_stock[0][0]
    end_of_year_price = one_stock[-1][0]
    if weights_dict[ticker] == 0:
        continue
    ratio = (end_of_year_price - start_of_year_price) / start_of_year_price
    total_real_profit += ratio * PORTFOLIO_CAPITAL * weights_dict[ticker]

final = (total_real_profit+PORTFOLIO_CAPITAL)

print("True returns of 2021 using Mean Variance method: {:.2f}%".format((final - PORTFOLIO_CAPITAL)/PORTFOLIO_CAPITAL*100))

True returns of 2021 using Mean Variance method: 51.21%


In [19]:

# weights_dict = dict(cleaned_weights)
weights_dict = dict(hrp_weights)
total_real_profit = 0
for ticker in weights_dict:
    one_stock = get_stock(ticker).values
    start_of_year_price = one_stock[0][0]
    end_of_year_price = one_stock[-1][0]
    if weights_dict[ticker] == 0:
        continue
    ratio = (end_of_year_price - start_of_year_price) / start_of_year_price
    total_real_profit += ratio * PORTFOLIO_CAPITAL * weights_dict[ticker]

final = (total_real_profit+PORTFOLIO_CAPITAL)

print("True returns of 2021 using HRP: {:.2f}%".format((final - PORTFOLIO_CAPITAL)/PORTFOLIO_CAPITAL*100))
    

True returns of 2021 using HRP: 40.09%


In [20]:

weights_dict = dict(cleaned_weights)
total_real_profit = 0
for ticker in weights_dict:
    one_stock = get_stock(ticker).values
    start_of_year_price = one_stock[0][0]
    end_of_year_price = one_stock[-1][0]
    if weights_dict[ticker] == 0:
        continue
    ratio = (end_of_year_price - start_of_year_price) / start_of_year_price
    total_real_profit += ratio * PORTFOLIO_CAPITAL * weights_dict[ticker]

final = (total_real_profit+PORTFOLIO_CAPITAL)

print("True returns of 2021 using cVar: {:.2f}%".format((final - PORTFOLIO_CAPITAL)/PORTFOLIO_CAPITAL*100))

True returns of 2021 using cVar: 45.68%
